# Tests

In [6]:
import autohf as hf
import autograd.numpy as anp
import autograd
import numpy as np
from autograd.differential_operators import make_jvp
import chemistry as chem
np.set_printoptions(linewidth=300)
from pennylane import qchem

In this Notebook, we implement many tests for the AutoHF differentiable Hartree-Fock solver. We will be using BeH$_2$ as our reference molecule.

In [7]:
# We define some basic information about the molecule
molecule = chem.BeH2()
R = anp.array([0.0, 1.0, 0.0, 0.0, -2.4359718264299817, 0.0, 0.0, 2.4359718264299817, 0.0]) # Optimal geometry
structure = ['Be', 'H', 'H']

charges = [4, 1, 1]
num_elecs = 6

R1, R2, R3 = anp.array(R[0:3]), anp.array(R[3:6]), anp.array(R[6:9]) # Atomic coordiantes

In [8]:
# We also define the basis set

basis_set = []
A1, A2, A3 = hf.basis_set_params("sto-3g", structure) # Gets default basis set parameters

for func in A1 + A2 + A3:
    L, exp, coeff = func
    basis_set.append(hf.AtomicBasisFunction(L, C=anp.array(coeff), A=anp.array(exp)))

In [9]:
num_act_orb = 6
num_act_elec = 4
wires = list(range(12))

core, active = qchem.active_space(6, 7, active_electrons=num_act_elec, active_orbitals=num_act_orb) # Prepares active space

### First Derivatives wrt Coordinates

In [10]:
def fd(func, vec, delta=0.00001):
    return lambda R : (1 / delta) * (func(R + (delta/2) * vec) - func(R - (delta/2) * vec))

In [11]:
# Testing the overlap matrix first derivative
from pennylane import qchem
num_act_orb = 6
num_act_elec = 4

core, active = qchem.active_space(6, 7, active_electrons=num_act_elec, active_orbitals=num_act_orb) # Prepares active space
fn = lambda r : hf.electron_integrals_flat(num_elecs, charges, basis_set, occupied=core, active=active)([r[0:3], r[3:6], r[6:9]], *([[r[0:3]]] * 5), [r[3:6]], [r[6:9]])
fn_jvp = make_jvp(fn)
vec = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0])

In [12]:
geometry = list(zip(structure, 1/chem.data.angs_bohr * np.array([R1, R2, R3])))
wires = list(range(12))
nuc_energy = 0.0
d = fn_jvp(R)(vec)[1]

core_ad, one_electron, two_electron = d[0], d[1:37].reshape((6, 6)), d[37:].reshape((6, 6, 6, 6))
dH = hf.build_h_from_integrals(geometry, one_electron, two_electron, nuc_energy, wires, basis="sto-3g", multiplicity=1, charge=0)

/Users/jackceroni/Library/Python/3.9/lib/python/site-packages/autohf-0.0.1-py3.9.egg/autohf/hartreefock.py:182: RuntimeWarning: divide by zero encountered in true_divide


In [13]:
otherdH = chem.d_hamiltonian(molecule, [basis_set[0:5], [basis_set[5]], [basis_set[6]]], 6, [4, 1, 1], wires, core=core, active=active)(R, vec)

  (1.0040806382940102) [I0]


In [124]:
print(d)

-0.7692516572147463


In [125]:
e = fd(fn, vec)(R)
print(e)

-0.7692516554413941


In [126]:
print(d - e)

-1.7733522428287074e-09


In [58]:
geometry1 = list(zip(structure, 1/chem.data.angs_bohr * np.array([R1 + np.array([0.0, 0.001, 0.0]), R2, R3])))
geometry2 = list(zip(structure, 1/chem.data.angs_bohr * np.array([R1 - np.array([0.0, 0.001, 0.0]), R2, R3])))
wires = list(range(12))
nuc_energy = 0.0

d1, d2 = fn(R + np.array([0.0, 0.001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])), fn(R - np.array([0.0, 0.001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]))
core_ad1, one_electron1, two_electron1 = d1[0], d1[1:37].reshape((6, 6)), d1[37:].reshape((6, 6, 6, 6))
core_ad2, one_electron2, two_electron2 = d2[0], d2[1:37].reshape((6, 6)), d2[37:].reshape((6, 6, 6, 6))

H_up = hf.build_h_from_integrals(geometry1, one_electron1, two_electron1, 0.0, wires, basis="sto-3g", multiplicity=1, charge=0)
H_down = hf.build_h_from_integrals(geometry2, one_electron2, two_electron2, 0.0, wires, basis="sto-3g", multiplicity=1, charge=0)
H = (1 / 0.002) * chem.accelerate_add(H_up, -1 * H_down)

IndexError: invalid index to scalar variable.

In [10]:
print(chem.accelerate_add(H, -1 * dH))

  (-5.551115123125783e-13) [Z10]
+ (-5.551115123125783e-13) [Z11]
+ (-3.3306690738754696e-13) [Z2]
+ (-1.97758476261356e-13) [Z9]
+ (-1.1102230246251565e-13) [Z3]
+ (-1.1102230246251565e-13) [Z7]
+ (-9.71445146547012e-14) [Z8]
+ (-5.551115123125783e-14) [Z6]
+ (-4.85722573273506e-14) [Z0]
+ (-4.163336342344337e-14) [Z1]
+ (2.220446049250313e-13) [Z5]
+ (2.498001805406602e-13) [Z4]
+ (4.440892098500626e-12) [I0]
+ (-1.203837815148179) [X8 X10]
+ (-1.203837815148179) [Y8 Y10]
+ (-2.220446049250313e-13) [Z10 Z11]
+ (-8.326672684688674e-14) [Z3 Z10]
+ (-8.326672684688674e-14) [Z2 Z11]
+ (-5.551115123125783e-14) [Z2 Z10]
+ (-5.551115123125783e-14) [Z3 Z11]
+ (-5.551115123125783e-14) [Z6 Z10]
+ (-5.551115123125783e-14) [Z6 Z11]
+ (-5.551115123125783e-14) [Z7 Z10]
+ (-5.551115123125783e-14) [Z7 Z11]
+ (-3.469446951953614e-14) [Z2 Z6]
+ (-3.469446951953614e-14) [Z3 Z7]
+ (-2.7755575615628914e-14) [Z3 Z6]
+ (-2.7755575615628914e-14) [Z2 Z7]
+ (-2.0816681711721685e-14) [Z6 Z7]
+ (-1.734723475976

In [49]:
fn = lambda r : hf.hf_energy(num_elecs, charges, basis_set)([r[0:3], r[3:6], r[6:9]], *([[r[0:3]]] * 5), [r[3:6]], [r[6:9]])

In [50]:
fd(fn, vec)(R)

0.33288867761882557

In [51]:
make_jvp(fn)(R)(vec)

(-8.4874876918407, 0.3328886772841439)